In [1]:
import pandas 
from collections import defaultdict

import mysql.connector
from sklearn.model_selection import train_test_split
import numpy as np
import time
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import KNNBaseline
from surprise import Dataset, Reader, AlgoBase
from EvaluationDataAdel import EvaluationDataAdel


from ContentKNN_Algo import ContentKNN_Algo

from mysql.connector import Error
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='elsupplier',
                                         user='root',
                                         password='')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        SQL_QUERY = "SELECT userId,productId, COUNT(*) as views_count FROM product_views GROUP BY  productId,userId ORDER BY `views_count`  DESC"
        cursor.execute(SQL_QUERY)
        product_views_result = cursor.fetchall()
        
        GET_PRODUCTS_QUERY ="SELECT products.id as product_id,"
        GET_PRODUCTS_QUERY +=" products.name as product_name,"
        GET_PRODUCTS_QUERY +=" products.supplierId as supplier_id,"
        GET_PRODUCTS_QUERY +=" suppliers.supplier_name,"
        GET_PRODUCTS_QUERY +=" products.subcategoryId as subcategory_id,"
        GET_PRODUCTS_QUERY +=" subcategories.name as subcategory_name,"
        GET_PRODUCTS_QUERY +=" categories.id as category_id,"
        GET_PRODUCTS_QUERY +=" categories.name as category_name"
        GET_PRODUCTS_QUERY +=" FROM `products` "
        GET_PRODUCTS_QUERY +=" INNER JOIN `suppliers` ON supplierId=suppliers.id "
        GET_PRODUCTS_QUERY +=" INNER JOIN `supplier_subcategories` ON products.subcategoryId=supplier_subcategories.id "
        GET_PRODUCTS_QUERY +=" INNER JOIN `subcategories` ON supplier_subcategories.subcategoryId=subcategories.id "
        GET_PRODUCTS_QUERY +=" INNER JOIN `categories` ON subcategories.categoryId=categories.id "
        
        
        cursor.execute(GET_PRODUCTS_QUERY)
        products_result = cursor.fetchall()
#         print("Your connected to database: ", record)
except Error as e:
    print("Error while connecting to MySQL", e)
# finally:
#     if (connection.is_connected()):
#         cursor.close()
#         connection.close()
#         print("MySQL connection is closed")


Connected to MySQL Server version  5.5.5-10.1.36-MariaDB


In [2]:
def getPopularityRanks():
    ratings = defaultdict(int)
    rankings = defaultdict(int)
    for row in product_views_result:
        product_id = int(row[0])
        ratings[product_id] += 1
        
    rank = 1
    for product_id, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
#         print(product_id, '--' , ratingCount)
        rankings[product_id] = rank
        rank += 1
    
    return rankings

In [3]:
rankings = getPopularityRanks()
# rankings

In [4]:
product_views_df = pandas.DataFrame(data=product_views_result)
# product_views_df
reader = Reader()
dataset = Dataset.load_from_df(product_views_df , reader)


In [5]:
product_views_df = pandas.DataFrame(data=product_views_result)
product_views_df.columns =["u_id", "p_id", "views"]
p_unique = product_views_df['p_id'].unique()
p_unique.shape

(6578,)

In [6]:
ed = EvaluationDataAdel(dataset, rankings)

DATAINEVDATA <surprise.dataset.DatasetAutoFolds object at 0x7f8a47192dd8>
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


test_ = ed.GetFullTrainSet()
test_.n_users
test_.n_items


In [7]:
products_result_df  = pandas.DataFrame(data=products_result) 
products_result_df.columns = ['product_id', 'product_name', 'supplier_id', 'supplier_name', 'subcategory_id', 'subcategory_name', 'category_id', 'category_name']

# products_result_df
categories = products_result_df['category_id'].unique()
categories


array([ 1,  7,  3,  2,  4, 14,  6,  5, 18, 20, 16, 21, 19, 22])

In [8]:
def getGenres():
    genres = defaultdict(list)
    genreIDs = {}
    maxGenreID = 0
    
    categories_len = len(categories)
    
    
    # iterate over rows with iterrows()
    for index, row in products_result_df.iterrows():
        product_categories_array =  [0] *categories_len
        p_id = row['product_id']
        category_id = row['category_id']
        index_in_categories = list(categories).index(category_id)
        product_categories_array[index_in_categories] = 1
        
        genres[p_id] = product_categories_array         
    
    
    return genres

# categories = getGenres()



In [9]:
# categories
# categories[168]


In [10]:
# getGenres()
# # Construct an Evaluator to, you know, evaluate them
# sim_options = {'name': 'cosine', 'user_based': False}
# algo = KNNBaseline(sim_options=sim_options)
# contentKNN = ContentKNN_Algo()
# # algo.setCategories('123')
# # fullTrainSet = ed.GetFullTrainSet()


# algo.fit(fullTrainSet, genes)
# trainSet = ed.GetFullTrainSet()

# trainSet.to_raw_iid(27)

In [11]:
def SampleTopNRecs(testSubject=1, k=10):
    print("\nUsing recommender ", "KNN")

    print("\nBuilding recommendation model...")
    
    contentKNN = ContentKNN_Algo()
    trainSet = ed.GetFullTrainSet()
    genes = getGenres()
    contentKNN.fit(trainSet, genes)
    print("Computing recommendations...")
    
    return contentKNN
#     recommendations = []

#     print ("\nWe recommend:")
#     for userID, movieID, actualRating, estimatedRating, _ in predictions:
#         intMovieID = int(movieID)
#         recommendations.append((intMovieID, estimatedRating))

#     recommendations.sort(key=lambda x: x[1], reverse=True)

#     for ratings in recommendations[:10]:
#         print(ml.getMovieName(ratings[0]), ratings[1])

In [12]:
model = SampleTopNRecs()


Using recommender  KNN

Building recommendation model...
trainset123 <surprise.trainset.Trainset object at 0x7f8a46ee7d68>
Computing content-based similarity matrix... 6578
similarities=index1 [0. 0. 0. ... 0. 0. 0.] len 6578
trainset????? <surprise.trainset.Trainset object at 0x7f8a46ee7d68>
0  of  6578
100  of  6578
200  of  6578
300  of  6578
400  of  6578
500  of  6578
600  of  6578
700  of  6578
800  of  6578
900  of  6578
1000  of  6578
1100  of  6578
1200  of  6578
1300  of  6578
1400  of  6578
1500  of  6578
1600  of  6578
1700  of  6578
1800  of  6578
1900  of  6578
2000  of  6578
2100  of  6578
2200  of  6578
2300  of  6578
2400  of  6578
2500  of  6578
2600  of  6578
2700  of  6578
2800  of  6578
2900  of  6578
3000  of  6578
3100  of  6578
3200  of  6578
3300  of  6578
3400  of  6578
3500  of  6578
3600  of  6578
3700  of  6578
3800  of  6578
3900  of  6578
4000  of  6578
4100  of  6578
4200  of  6578
4300  of  6578
4400  of  6578
4500  of  6578
4600  of  6578
4700  of  65

In [13]:
def predict(model,userId = 0):
    testSet = ed.GetAntiTestSetForUser(userId)
    predictions = model.test(testSet)
    
    recommendations = []

    print ("\nWe recommend:")
    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        intMovieID = int(movieID)
        recommendations.append((intMovieID, estimatedRating))

    recommendations.sort(key=lambda x: x[1], reverse=True)
    # print('recommendations' ,recommendations)

   
    #     print(ml.getMovieName(ratings[0]), ratings[1])


    return recommendations[0:10]

In [14]:
row_user_id = 428
recommendations = predict(model,row_user_id)
for ratings in recommendations:
    print('rating', ratings, products_result[ratings[0]][1])

print("User ",row_user_id," Recommends=>",recommendations)

trainset <surprise.trainset.Trainset object at 0x7f8a46ee7d68>
user=  428

We recommend:
rating (168, 5) Nescafe Alegria Intense - اليجرية نسكافية
rating (7022, 5) Nescafe Classic - نسكافيه كلاسيك
rating (7722, 5) وقار بلدى 
rating (123, 5) حلاوة أركان 750غ سادة 
rating (8500, 5) None
rating (5683, 5) Ice Cream -  ايس كريم 
rating (7425, 5) Brownies
rating (5670, 5) Artisan Baguette Bread خبز باجيت ارتيزان 
rating (213, 5) NESCAFE Classic 12x150gm
rating (831, 5) Premium Smoked Octopus- أخطبوط مدخن 
User  428  Recommends=> [(168, 5), (7022, 5), (7722, 5), (123, 5), (8500, 5), (5683, 5), (7425, 5), (5670, 5), (213, 5), (831, 5)]


In [22]:
model.predict(397, 123)

Prediction(uid=397, iid=123, r_ui=None, est=1.25, details={'was_impossible': False})

,uid,iid,rating
0,0.0,0.0,117.0
1,0.0,1.0,109.0
2,0.0,2.0,92.0
3,0.0,3.0,69.0
4,0.0,4.0,68.0
5,0.0,5.0,66.0
6,0.0,6.0,65.0
7,0.0,7.0,64.0
8,0.0,8.0,64.0
9,0.0,9.0,47.0
